In [1]:
!pip install tensorflow keras_tuner --upgrade



In [2]:
#1 Importing Important Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt  # latest KerasTuner

In [3]:
#2 Uploading The DataSet
from google.colab import files
uploaded = files.upload()



Saving Real_Combine.csv to Real_Combine (1).csv


In [4]:
#3 To display all the columns in one line
pd.set_option('display.max_columns', None)   # Saare columns dikhayega
pd.set_option('display.width', 1000)        # Output ek single line mein aayega


In [7]:
#4 Uploading the Data Set
df = pd.read_csv('Real_Combine (1).csv') # Apni CSV ka path sahi likho
print(df.head()) # Data ko inspect karo

# Check NaN values
print('NaN in features:', df.isna().sum())

# Drop or fill NaN
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

      T    TM   Tm     SLP     H   VV    V    VM      PM 2.5
0   7.4   9.8  4.8  1017.6  93.0  0.5  4.3   9.4  219.720833
1   7.8  12.7  4.4  1018.5  87.0  0.6  4.4  11.1  182.187500
2   6.7  13.4  2.4  1019.4  82.0  0.6  4.8  11.1  154.037500
3   8.6  15.5  3.3  1018.7  72.0  0.8  8.1  20.6  223.208333
4  12.4  20.9  4.4  1017.3  61.0  1.3  8.7  22.2  200.645833
NaN in features: T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=0)


In [9]:
#6 Tunning the model by using keras tunner
def build_model(hp):
    model = keras.Sequential()
    # Tune number of layers between 1 and 3 (regression ke liye zyada layers avoid karo)
    for i in range(hp.Int('num_layers', 1, 3)):
        # Tune units per layer between 32 and 128
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
            activation='relu'
        ))
    model.add(layers.Dense(1, activation='linear')) # Regression output
    # Tune learning rate
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )
    return model


In [10]:
#7 Hyperparameter Search Setup
tuner = kt.RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5, # Zyada trials zyada time le sakte hain
    executions_per_trial=2,
    directory='project',
    project_name='AirQualityIndex',
    overwrite=True
)




In [11]:
#9 Hyperparameter Tunning Run
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Trial 5 Complete [00h 00m 11s]
val_mean_absolute_error: 83.28109741210938

Best val_mean_absolute_error So Far: 37.9595832824707
Total elapsed time: 00h 00m 52s


In [12]:
tuner.results_summary()


Results summary
Results in project/AirQualityIndex
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 64
learning_rate: 0.01
units_1: 128
Score: 37.9595832824707

Trial 2 summary
Hyperparameters:
num_layers: 2
units_0: 64
learning_rate: 0.001
units_1: 32
Score: 46.48422431945801

Trial 4 summary
Hyperparameters:
num_layers: 3
units_0: 128
learning_rate: 0.0001
units_1: 96
units_2: 128
Score: 83.28109741210938

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 32
learning_rate: 0.0001
units_1: 128
units_2: 32
Score: 99.7461051940918

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 32
learning_rate: 0.0001
units_1: 32
Score: 104.94769287109375


In [13]:
#best model
best_model = tuner.get_best_models(num_models=1)[0]
loss, mae = best_model.evaluate(X_test, y_test)
print('Test MAE:', mae)


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 38.1068 - mean_absolute_error: 38.1068
Test MAE: 37.92344665527344
